In [ ]:
import xarray as xr

import xdggs

_ = xr.set_options(display_expand_data=False)

# Working with H3 data

H3 is a popular icosahedral DGGS with hexagonal cells, developed and popularized by Uber. For more information, see https://h3geo.org

## initialization

To initialize, we first have to open the dataset. Here we'll use `xarray`'s `air_temperature` tutorial dataset, which was interpolated to the H3 grid.

```{tip}
If the dataset you want to work on is not already on a H3 grid, you will have to use a different package to interpolate.
```

```{warning}
For the purpose of this tutorial we drop the geographic coordinates and load all data into memory, but this is not required.
```

In [ ]:
original_ds = xdggs.tutorial.open_dataset("air_temperature", "h3").load()
air_temperature = original_ds.drop_vars(["lat", "lon"])
air_temperature

After that, we can use {py:func}`xdggs.decode` to tell `xdggs` to interpret the cell ids.

This will create a grid object (see {py:attr}`xarray.Dataset.dggs.grid_info`) containing the grid parameters and a custom index for the `cell_ids` coordinate (notice how the coordinate name is displayed in bold?), which will allow us to perform grid-aware operations.

```{important}
For this to work, the dataset has to have a coordinate called `cell_ids`, and it also has to have the `grid_name` attribute and exactly one of the `resolution` or `level` attributes.

In this case, `grid_name` is `"h3"`, and the `resolution` is 2 (the `h3` libraries call this the "cell resolution", while `xdggs` will always refer to `level`).
```

In [ ]:
ds = air_temperature.pipe(xdggs.decode)
ds

## deriving data

With the grid object and the custom index, we can derive additional data from the cell ids.

### cell center coordinates

For example, we can reconstruct the cell centers we dropped from the original dataset, using {py:meth}`xarray.Dataset.dggs.cell_centers`:

In [ ]:
cell_centers = ds.dggs.cell_centers()
cell_centers

these are the same as the ones we dropped before:

In [ ]:
derived_ds = ds.assign_coords(
    cell_centers.rename_vars({"latitude": "lat", "longitude": "lon"}).coords
)
derived_ds

In [ ]:
xr.testing.assert_equal(derived_ds, original_ds)

### cell boundary polygons

Additionally, we can derive the cell boundary polygons as an array of {doc}`shapely:index` using {py:meth}`xarray.Dataset.dggs.cell_boundaries`:

In [ ]:
cell_boundaries = ds.dggs.cell_boundaries()
cell_boundaries

## plotting

We can quickly visualize the data using {py:meth}`xarray.DataArray.dggs.explore`, which is powered by [lonboard](https://github.com/developmentseed/lonboard).

```{warning}
This is currently restricted to 1D `DataArray` objects, so we need to select a single timestep.
```

In [ ]:
ds["air"].isel(time=15).dggs.explore()